In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import helpers.processing_helpers as ph

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

In [101]:
df_dev = pd.read_csv("./dataset/development.csv")

In [102]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_dev.drop(columns=ph.get_column_names(features, noise_indexes))
sensors_removed = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))

In [103]:
sample = sensors_removed.sample(20000)

In [104]:
df_pmax = sample[[f'pmax[{i}]' for i in acc_idxs]]
df_negpmax = sample[[f'negpmax[{i}]' for i in acc_idxs]]
df_area = sample[[f'area[{i}]' for i in acc_idxs]]

In [105]:
mask = df_pmax < 20
df_pmax[mask] = 0

C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\1029706993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pmax[mask] = 0
C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\1029706993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pmax[mask] = 0


In [106]:
mask.columns = ph.get_column_names(['negpmax'], acc_idxs)
df_negpmax[mask] = 0

C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\3468149505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negpmax[mask] = 0
C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\3468149505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negpmax[mask] = 0


In [107]:
mask.columns = ph.get_column_names(['area'], acc_idxs)
df_area[mask] = 0

C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\3210989202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area[mask] = 0
C:\Users\Arist\AppData\Local\Temp\ipykernel_7876\3210989202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area[mask] = 0


In [108]:
df_zero = pd.concat([df_pmax, df_negpmax, df_area], axis=1)

In [109]:
y_train_valid = sample[['x', 'y']].copy()

X_train_valid = sample.drop(columns=['x', 'y'])

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

reg = RandomForestRegressor(100, random_state=42, verbose=1)
reg.fit(X_train , y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  2.6min


RandomForestRegressor(random_state=42, verbose=1)

In [110]:
med_without_noise = ph.mean_euclid_dist(y_valid, reg.predict(X_valid))
print(med_without_noise)

5.865225359452209


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [111]:
X_train, X_valid, y_train, y_valid = train_test_split(df_zero, y_train_valid, shuffle=True, random_state=42)

reg_zero = RandomForestRegressor(100, random_state=42, verbose=1)
reg_zero.fit(X_train , y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   54.2s


RandomForestRegressor(random_state=42, verbose=1)

In [112]:
med = ph.mean_euclid_dist(y_valid, reg_zero.predict(X_valid))
print(med)

5.602707308641181


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
